In [1]:
import sys
sys.path.append('..')
import numpy as np
import pandas as pd
from virtualscada.vs import removeRows
from virtualscada.vs import fillValuesMLPFForward

In [2]:
p = pd.read_csv('C:/Users/Serhan/PycharmProjects/virtual-scada/Data/output_p.csv', header = None)
q = pd.read_csv('C:/Users/Serhan/PycharmProjects/virtual-scada/Data/output_q.csv', header = None)
v = pd.read_csv('C:/Users/Serhan/PycharmProjects/virtual-scada/Data/output_v.csv', header = None)
a = pd.read_csv('C:/Users/Serhan/PycharmProjects/virtual-scada/Data/output_a.csv', header = None)

In [3]:
q = pd.DataFrame(np.radians(q.values))

In [4]:
pRemoved = removeRows(p, .1, colPercentage = .5)
qRemoved = removeRows(q, .1, colPercentage = .5)

..\virtualscada\vs.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dataRemoved[i][j] = None


In [ ]:
pFilled, qFilled = fillValuesMLPFForward(pRemoved, qRemoved, v, a, max_iter = -1)


C:\Users\Serhan\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [ ]:
def getRMSE(OriginalData, removedData, filledData):
    numRows, numCols = OriginalData.shape
    predictions = []
    actuals = []
    for i in range(numCols):
        for j in range(numRows):
            if np.isnan(removedData[i][j]):
                predictions.append(filledData[i][j])
                actuals.append(OriginalData[i][j])
                
    return np.sqrt(np.nanmean((np.array(predictions)-np.array(actuals))**2))

In [19]:
print(getRMSE(p, pRemoved, pFilled))

2425527.9132901826


In [19]:
def nonNullIntersection(dataList):
    """
    Takes a list of dataframes and returns the list of dataframes with rows that contain a null value in
    any dataframe removed. This is so that the row can be used for training ML Powerflow
    """

    masks = []
    for data in dataList:
        mask = data.isna().any(axis=1)
        masks.append(~mask)

    finalMask = masks[0]
    for mask in masks[1:]:
        finalMask = finalMask & mask

    for x in range(len(dataList)):
        dataList[x] = dataList[x][finalMask]

    return dataList

In [20]:
dl = nonNullIntersection([pRemoved,qRemoved,v,a])

In [21]:
dl[1]

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.382419,-0.074368,-0.002161,-0.004135,-0.234991,1.399876e-10,-0.051402,-0.082823,3.548429e-11,-0.021794,...,-0.021050,0.185843,-0.048475,-0.021918,4.467694e-12,-0.004094,0.010447,1.806765e-10,-0.005352,-0.009072
1,0.406801,-0.095663,-0.001327,-0.007421,-0.218376,1.754754e-10,-0.067042,-0.071599,4.243244e-11,-0.021763,...,-0.022955,0.172402,-0.060851,-0.018225,4.399626e-12,-0.004087,0.012517,2.059139e-10,-0.005916,-0.009918
2,0.410484,-0.092548,-0.001327,-0.007978,-0.217590,2.267881e-10,-0.081545,-0.059411,4.905771e-11,-0.029732,...,-0.020712,0.181130,-0.069260,-0.026482,4.414636e-12,-0.004094,0.014330,2.348166e-10,-0.005491,-0.009838
4,0.405231,-0.101618,-0.001327,-0.004908,-0.208672,1.883559e-10,-0.071347,-0.064860,4.806986e-11,-0.025569,...,-0.024004,0.184150,-0.070443,-0.019761,4.544837e-12,-0.004094,0.011374,2.162637e-10,-0.006650,-0.009058
5,0.419891,-0.099522,-0.001333,-0.002819,-0.215670,2.468768e-10,-0.067541,-0.068255,6.367310e-11,-0.021794,...,-0.024180,0.168651,-0.080268,-0.019921,5.039813e-12,-0.004101,0.013808,2.681000e-10,-0.006594,-0.009101
6,0.438008,-0.086706,-0.002218,-0.005723,-0.230104,3.484026e-10,-0.075695,-0.073971,8.092917e-11,-0.023915,...,-0.023564,0.175458,-0.089694,-0.017255,6.338687e-12,-0.004101,0.017986,3.433237e-10,-0.005206,-0.009984
7,0.411357,-0.040532,-0.002212,-0.006642,-0.256040,3.396760e-10,-0.055323,-0.113267,7.587644e-11,-0.022654,...,-0.023564,0.182911,-0.089682,-0.015720,6.234840e-12,-0.004101,0.029250,3.474776e-10,-0.005859,-0.012762
9,0.430957,-0.077852,-0.002161,-0.002840,-0.250088,2.713987e-10,-0.041314,-0.100576,7.176794e-11,-0.027273,...,-0.025888,0.180973,-0.087757,-0.019558,5.897118e-12,-0.004884,0.021824,3.102672e-10,-0.004687,-0.012135
10,0.382087,-0.040088,-0.001333,-0.002826,-0.270596,2.422866e-10,-0.041897,-0.103694,5.658707e-11,-0.021827,...,-0.027164,0.173512,-0.087361,-0.020827,2.746974e-12,-0.005147,0.072045,1.544634e-10,-0.040017,-0.008547
11,0.409175,-0.028194,-0.001333,-0.002994,-0.301715,3.761883e-10,-0.039787,-0.070426,7.793070e-11,-0.022028,...,-0.027119,0.176208,-0.085781,-0.019549,5.831494e-12,-0.004848,0.018539,3.602709e-10,-0.005643,-0.010246


In [22]:
q

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.382419,-0.074368,-0.002161,-0.004135,-0.234991,1.399876e-10,-0.051402,-0.082823,3.548429e-11,-0.021794,...,-0.021050,0.185843,-0.048475,-0.021918,4.467694e-12,-0.004094,0.010447,1.806765e-10,-0.005352,-0.009072
1,0.406801,-0.095663,-0.001327,-0.007421,-0.218376,1.754754e-10,-0.067042,-0.071599,4.243244e-11,-0.021763,...,-0.022955,0.172402,-0.060851,-0.018225,4.399626e-12,-0.004087,0.012517,2.059139e-10,-0.005916,-0.009918
2,0.410484,-0.092548,-0.001327,-0.007978,-0.217590,2.267881e-10,-0.081545,-0.059411,4.905771e-11,-0.029732,...,-0.020712,0.181130,-0.069260,-0.026482,4.414636e-12,-0.004094,0.014330,2.348166e-10,-0.005491,-0.009838
3,0.412962,-0.078772,-0.004013,-0.005611,-0.235043,2.684665e-10,-0.074393,-0.059243,5.834985e-11,-0.021649,...,-0.022986,0.174847,-0.073217,-0.018799,4.992863e-12,-0.004101,0.013882,2.701944e-10,-0.005631,-0.009823
4,0.405231,-0.101618,-0.001327,-0.004908,-0.208672,1.883559e-10,-0.071347,-0.064860,4.806986e-11,-0.025569,...,-0.024004,0.184150,-0.070443,-0.019761,4.544837e-12,-0.004094,0.011374,2.162637e-10,-0.006650,-0.009058
5,0.419891,-0.099522,-0.001333,-0.002819,-0.215670,2.468768e-10,-0.067541,-0.068255,6.367310e-11,-0.021794,...,-0.024180,0.168651,-0.080268,-0.019921,5.039813e-12,-0.004101,0.013808,2.681000e-10,-0.006594,-0.009101
6,0.438008,-0.086706,-0.002218,-0.005723,-0.230104,3.484026e-10,-0.075695,-0.073971,8.092917e-11,-0.023915,...,-0.023564,0.175458,-0.089694,-0.017255,6.338687e-12,-0.004101,0.017986,3.433237e-10,-0.005206,-0.009984
7,0.411357,-0.040532,-0.002212,-0.006642,-0.256040,3.396760e-10,-0.055323,-0.113267,7.587644e-11,-0.022654,...,-0.023564,0.182911,-0.089682,-0.015720,6.234840e-12,-0.004101,0.029250,3.474776e-10,-0.005859,-0.012762
8,0.443331,-0.100501,-0.005150,-0.003558,-0.233996,2.957111e-10,-0.056449,-0.071129,7.907214e-11,-0.022630,...,-0.026285,0.169271,-0.091328,-0.015267,5.857674e-12,-0.004962,0.019295,3.049788e-10,-0.005631,-0.012704
9,0.430957,-0.077852,-0.002161,-0.002840,-0.250088,2.713987e-10,-0.041314,-0.100576,7.176794e-11,-0.027273,...,-0.025888,0.180973,-0.087757,-0.019558,5.897118e-12,-0.004884,0.021824,3.102672e-10,-0.004687,-0.012135


In [ ]:
print(1e3)